# Scraping Travel Costs

In [ ]:
The last notebook scraped

In [1]:
import wget
import requests
import pandas as pd
import pytesseract
from pdf2image import convert_from_path
from Andrew import sandwich
import numpy as np
import os
import time
import fitz
import re
import glob
from pathlib import Path
import re

In [2]:
df=pd.read_csv("International Travel Jan 10.csv")
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Page #,Departure,Return,Year,Country
0,0,0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE
1,1,1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE
2,2,2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE
3,3,3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE
4,4,4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,1,"Dec. 7, 2023","Dec. 11, 2023",2023,UAE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,6195,4,"Nov. 8, 2018 - Nov. 13, 2018",Carlos Curbelo,Carlos Curbelo,R,FL-26,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France
1629,6196,5,"Nov. 8, 2018 - Nov. 13, 2018",Peter Welch,Peter Welch,D,VT-1,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France
1630,6197,6,"Nov. 7, 2018 - Nov. 12, 2018",Mike Doyle,Mike Doyle,D,PA-0,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France
1631,6198,7,"Nov. 7, 2018 - Nov. 12, 2018",Josh Gottheimer,Josh Gottheimer,D,NJ-5,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France


In [3]:
df["Raw Costs"]=""
df=df.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
df


,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Page #,Departure,Return,Year,Country,Raw Costs
0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE,
1,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 12, 2023",2023,UAE,
2,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE,
3,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,1,"Dec. 6, 2023","Dec. 11, 2023",2023,UAE,
4,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,1,"Dec. 7, 2023","Dec. 11, 2023",2023,UAE,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1628,"Nov. 8, 2018 - Nov. 13, 2018",Carlos Curbelo,Carlos Curbelo,R,FL-26,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France,
1629,"Nov. 8, 2018 - Nov. 13, 2018",Peter Welch,Peter Welch,D,VT-1,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 8, 2018","Nov. 13, 2018",2018,France,
1630,"Nov. 7, 2018 - Nov. 12, 2018",Mike Doyle,Mike Doyle,D,PA-0,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France,
1631,"Nov. 7, 2018 - Nov. 12, 2018",Josh Gottheimer,Josh Gottheimer,D,NJ-5,"Paris, France",Ripon Society and the Franklin Center for Glob...,Original,https://disclosures-clerk.house.gov/gtimages/M...,152,"Nov. 7, 2018","Nov. 12, 2018",2018,France,


In [5]:
len(df)

1633

In [4]:
#Download and scrape the pdf of a privately-sponsored congressional trip
def read_housefiles(file):
    '''
    Takes a house file(pdf) and reads the entries for traveler cost and accompanying 
    family member cost. Returns "error" if the file cannot be read.
    '''
    print(f"Reading{file}\n")
    doc=convert_from_path(file)
    costs=[]
    print(f"now scraping {file}")
    for page in doc: 
        text=pytesseract.image_to_string(page)
        lower=text.lower()
        if (seektext1 in lower) and (seektext2 in lower):
            costs=sandwich(text, start_text, end_text)
            if costs!="Error making sandwich":
                print(f"trip cost {costs}")
                return costs
                break
            else:
                print(f"unable to scrape costs in {file}")
                return costs
        else:
            pass
    if costs==[]:
        return "error"

In [5]:
seektext1="actual amount of expenses paid on behalf of"
seektext2="sponsor post-travel disclosure form"
start_text="and description"
end_text="Family Member"

In [6]:
def fitz_read(file):
    doc=fitz.open(file)
    costs=[]
    for page in doc:
        text=page.get_text()
        lower=text.lower()
        if(seektext1 in lower) and (seektext2 in lower):
            costs=sandwich(text, start_text,end_text)
            if costs!="Error making sandwich":
                print(f"trip cost {costs}")
                return costs
                break
            else:
                print(f"unable to scrape costs in {file}")
                return costs
        else:
            pass
    if costs==[]:
        return "error"

In [5]:
def cleancosts(string):
    '''
    Takes the string returned by the previous function. Attempts to strip out dollar signs
    and other mischellanea to add up the total cost of a trip. 
    If there's still leftover text, it returns the original string
    '''
    string=str(string)
    newstring=string.strip().replace("Traveler ","").replace("|","").replace("/","").replace("\\","")
    newstring=newstring.replace(" Accompanying ","").replace("1$","")
    newstring=newstring.replace("Accompanying ","")
    newstring=newstring.replace("$","").replace(",","").replace("\n","")
    numblist=newstring.split(" ")
    try:
        numbers=[float(x) for x in numblist]
        return sum(numbers)
    except: 
        string=string

In [9]:
broken_links=[]
costs_dict=[]
no_forms=[]
for i in range(55, 1634):
    row=df.loc[i]
    raw_costs=[]
    fitz_costs=[]
    destination=row['Country']
        #docfile=row["hrefs"]
    url=row["hrefs"]
    if destination!="USA":
        filename=f"Downloads/{i}{destination}{row['Dates']}.pdf"
        response=requests.get(url)
        if response.status_code<400:
            print(f"now downloading information on {row['Traveler']}'s trip to {destination}' on {row['Dates']}")
            wget.download(url, out=filename)
            raw_costs=read_housefiles(filename)
            if raw_costs=="error":
                no_forms.append([i, row['Traveler'], row['Dates'],destination])
                print(f"No disclosure found in row {i}")
            else:
                print(f"{row['Traveler']}'s trip cost:\n {raw_costs}\n")
            df.loc[i,"Raw Costs"]=raw_costs
            os.remove(filename)
        else:
            print(f"failed to read {row['Traveler']} on {row['Dates']}")
            broken_links.append({"row":i, "traveler":row['Traveler'],"Date":row["Dates"],"url":url})
    else:
        pass
    time.sleep(1)
print("Scraping complete")

now downloading information on William M. "Bill" Christian's trip to France' on Sept. 5, 2023 - Sept. 8, 2023
ReadingDownloads/55FranceSept. 5, 2023 - Sept. 8, 2023.pdf

now scraping Downloads/55FranceSept. 5, 2023 - Sept. 8, 2023.pdf
unable to scrape costs in Downloads/55FranceSept. 5, 2023 - Sept. 8, 2023.pdf
William M. "Bill" Christian's trip cost:
 Error making sandwich

now downloading information on Randy Weber, Sr.'s trip to France' on Sept. 5, 2023 - Sept. 8, 2023
ReadingDownloads/56FranceSept. 5, 2023 - Sept. 8, 2023.pdf

now scraping Downloads/56FranceSept. 5, 2023 - Sept. 8, 2023.pdf
trip cost )

oe . mn PE

Accompanying |N/A

Randy Weber, Sr.'s trip cost:
 )

oe . mn PE

Accompanying |N/A


now downloading information on Chellie Pingree's trip to Norway' on Aug. 26, 2023 - Sept. 5, 2023
ReadingDownloads/57NorwayAug. 26, 2023 - Sept. 5, 2023.pdf

now scraping Downloads/57NorwayAug. 26, 2023 - Sept. 5, 2023.pdf
trip cost )
Traveler $1,075 $280 $692: AV, meeting
$4,411 (See Ap

ReadingDownloads/73LithuaniaAug. 27, 2023 - Sept. 2, 2023.pdf

now scraping Downloads/73LithuaniaAug. 27, 2023 - Sept. 2, 2023.pdf
trip cost )

Traveler $2387 05 $8.26 Museum of Occupati
: . pation
$423 $502.31 and Freedom Fights tour
$246 Private bus transport

Accompanying |N/A N/A N/A N/A

Leslie Shedd's trip cost:
 )

Traveler $2387 05 $8.26 Museum of Occupati
: . pation
$423 $502.31 and Freedom Fights tour
$246 Private bus transport

Accompanying |N/A N/A N/A N/A


now downloading information on Gary J. Palmer's trip to Norway' on Aug. 26, 2023 - Sept. 2, 2023
ReadingDownloads/74NorwayAug. 26, 2023 - Sept. 2, 2023.pdf

now scraping Downloads/74NorwayAug. 26, 2023 - Sept. 2, 2023.pdf
unable to scrape costs in Downloads/74NorwayAug. 26, 2023 - Sept. 2, 2023.pdf
Gary J. Palmer's trip cost:
 Error making sandwich

now downloading information on Jesse Von Stein's trip to Armenia Azerbaijan' on Aug. 26, 2023 - Sept. 2, 2023
ReadingDownloads/75Armenia AzerbaijanAug. 26, 2023 - Sept. 2, 2

now downloading information on Kevin Diamond's trip to Lithuania' on Aug. 27, 2023 - Sept. 1, 2023
ReadingDownloads/91LithuaniaAug. 27, 2023 - Sept. 1, 2023.pdf

now scraping Downloads/91LithuaniaAug. 27, 2023 - Sept. 1, 2023.pdf
trip cost )
Traveler $2387.05 $423 $453.43 $8.26 Museum of
Occupation and Freedom
Fights tour
$246 Private bus transport
Accompanying |N/A N/A N/A N/A

Kevin Diamond's trip cost:
 )
Traveler $2387.05 $423 $453.43 $8.26 Museum of
Occupation and Freedom
Fights tour
$246 Private bus transport
Accompanying |N/A N/A N/A N/A


now downloading information on Kim Schrier's trip to Norway' on Aug. 26, 2023 - Sept. 1, 2023
ReadingDownloads/92NorwayAug. 26, 2023 - Sept. 1, 2023.pdf

now scraping Downloads/92NorwayAug. 26, 2023 - Sept. 1, 2023.pdf
trip cost )
Traveler $1,075 $280 $692: AV, meeting
$6,990 (See Appendix) | (See Appendix) | rooms, conference
services
Accompanying

Kim Schrier's trip cost:
 )
Traveler $1,075 $280 $692: AV, meeting
$6,990 (See Appendix) | (See

ReadingDownloads/112El SavadorAug. 22, 2023 - Aug. 25, 2023.pdf

now scraping Downloads/112El SavadorAug. 22, 2023 - Aug. 25, 2023.pdf
trip cost )
$1619.75 $403.00 $300
Traveler
Accompanying

J. Luis Correa's trip cost:
 )
$1619.75 $403.00 $300
Traveler
Accompanying


now downloading information on Greg Casar's trip to Brazil' on Aug. 13, 2023 - Aug. 24, 2023
ReadingDownloads/113BrazilAug. 13, 2023 - Aug. 24, 2023.pdf

now scraping Downloads/113BrazilAug. 13, 2023 - Aug. 24, 2023.pdf
trip cost )

Interpretation: $374
$4,546 $1 1380 $672 Photography: $384
Hotel Conference Rooms: $98
M&IE: $55

Traveler

Accompanying

Greg Casar's trip cost:
 )

Interpretation: $374
$4,546 $1 1380 $672 Photography: $384
Hotel Conference Rooms: $98
M&IE: $55

Traveler

Accompanying


now downloading information on Greg Casar's trip to Brazil' on Aug. 13, 2023 - Aug. 24, 2023
ReadingDownloads/114BrazilAug. 13, 2023 - Aug. 24, 2023.pdf

now scraping Downloads/114BrazilAug. 13, 2023 - Aug. 24, 2023.pdf
trip 

trip cost )

Traveler

Interpretation: $374
$4,955 $1 945 $813 Photography: $384
Hotel Conference Rooms: $98
Accompanying

Alexandria Ocasio-Cortez's trip cost:
 )

Traveler

Interpretation: $374
$4,955 $1 945 $813 Photography: $384
Hotel Conference Rooms: $98
Accompanying


now downloading information on Ami Bera's trip to Kenya' on Aug. 12, 2023 - Aug. 21, 2023
ReadingDownloads/132KenyaAug. 12, 2023 - Aug. 21, 2023.pdf

now scraping Downloads/132KenyaAug. 12, 2023 - Aug. 21, 2023.pdf
trip cost )

$1,844: AV, mtg.

$10,783 space, visa, security,
, conference services

Accompanying $0 $288 $1,844: AV, mtg.

Ami Bera's trip cost:
 )

$1,844: AV, mtg.

$10,783 space, visa, security,
, conference services

Accompanying $0 $288 $1,844: AV, mtg.


now downloading information on Maxwell Frost's trip to Brazil' on Aug. 13, 2023 - Aug. 21, 2023
ReadingDownloads/133BrazilAug. 13, 2023 - Aug. 21, 2023.pdf

now scraping Downloads/133BrazilAug. 13, 2023 - Aug. 21, 2023.pdf
unable to scrape costs i

...

now downloading information on Jeff Fortenberry's trip to Kenya' on March 15, 2019 - March 22, 2019
ReadingDownloads/1545KenyaMarch 15, 2019 - March 22, 2019.pdf

now scraping Downloads/1545KenyaMarch 15, 2019 - March 22, 2019.pdf
trip cost 

$6190.00 $982.75 $313.00 $653.00
Tips, Conservation fees,
park fees

Accompanying $6139.30 $982.75 $313.00 $653.00

Jeff Fortenberry's trip cost:
 

$6190.00 $982.75 $313.00 $653.00
Tips, Conservation fees,
park fees

Accompanying $6139.30 $982.75 $313.00 $653.00


now downloading information on Gus M. Bilirakis's trip to Turkey' on March 14, 2019 - March 19, 2019
ReadingDownloads/1546TurkeyMarch 14, 2019 - March 19, 2019.pdf

now scraping Downloads/1546TurkeyMarch 14, 2019 - March 19, 2019.pdf
trip cost 
Gus M. Bilirakis's trip cost:
 

now downloading information on Elizabeth Hittos's trip to Turkey' on March 14, 2019 - March 19, 2019
ReadingDownloads/1547TurkeyMarch 14, 2019 - March 19, 2019.pdf

now scraping Downloads/1547TurkeyMarch 14, 2019 

trip cost 

$11134.15 $1336.8 $700.00 $696.72 (interpreter,
room rental fee, local

guide, bag shipping fee)

Accompanying

Billy Long's trip cost:
 

$11134.15 $1336.8 $700.00 $696.72 (interpreter,
room rental fee, local

guide, bag shipping fee)

Accompanying


now downloading information on James Johnson's trip to Armenia' on Feb. 16, 2019 - Feb. 23, 2019
ReadingDownloads/1568ArmeniaFeb. 16, 2019 - Feb. 23, 2019.pdf

now scraping Downloads/1568ArmeniaFeb. 16, 2019 - Feb. 23, 2019.pdf
trip cost 

$636.18 s $73.03 (meeting rooms)
$450.01 (interpretation)

Accompanying |N/A N/A

James Johnson's trip cost:
 

$636.18 s $73.03 (meeting rooms)
$450.01 (interpretation)

Accompanying |N/A N/A


now downloading information on Erica Riordan's trip to Armenia' on Feb. 16, 2019 - Feb. 23, 2019
ReadingDownloads/1569ArmeniaFeb. 16, 2019 - Feb. 23, 2019.pdf

now scraping Downloads/1569ArmeniaFeb. 16, 2019 - Feb. 23, 2019.pdf
trip cost 
Erica Riordan's trip cost:
 

now downloading information on A

now downloading information on Debbie Lesko's trip to United Kingdom' on Dec. 13, 2018 - Dec. 18, 2018
ReadingDownloads/1589United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf

now scraping Downloads/1589United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf
No disclosure found in row 1589
now downloading information on Jenniffer Gonzalez-Colon's trip to United Kingdom' on Dec. 13, 2018 - Dec. 18, 2018
ReadingDownloads/1590United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf

now scraping Downloads/1590United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf
No disclosure found in row 1590
now downloading information on Michael R. Turner's trip to United Kingdom' on Dec. 13, 2018 - Dec. 18, 2018
ReadingDownloads/1591United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf

now scraping Downloads/1591United KingdomDec. 13, 2018 - Dec. 18, 2018.pdf
No disclosure found in row 1591
now downloading information on Samantha Schifrin's trip to Moldova' on Dec. 10, 2018 - Dec. 16, 2018
ReadingDownloads/1592MoldovaDec. 10, 2018 

ReadingDownloads/1614FranceNov. 8, 2018 - Nov. 13, 2018.pdf

now scraping Downloads/1614FranceNov. 8, 2018 - Nov. 13, 2018.pdf
unable to scrape costs in Downloads/1614FranceNov. 8, 2018 - Nov. 13, 2018.pdf
Bill Pascrell's trip cost:
 Error making sandwich

now downloading information on Frank Lucas's trip to France' on Nov. 8, 2018 - Nov. 13, 2018
ReadingDownloads/1615FranceNov. 8, 2018 - Nov. 13, 2018.pdf

now scraping Downloads/1615FranceNov. 8, 2018 - Nov. 13, 2018.pdf
unable to scrape costs in Downloads/1615FranceNov. 8, 2018 - Nov. 13, 2018.pdf
Frank Lucas's trip cost:
 Error making sandwich

now downloading information on Rodney Davis's trip to France' on Nov. 8, 2018 - Nov. 13, 2018
ReadingDownloads/1616FranceNov. 8, 2018 - Nov. 13, 2018.pdf

now scraping Downloads/1616FranceNov. 8, 2018 - Nov. 13, 2018.pdf
unable to scrape costs in Downloads/1616FranceNov. 8, 2018 - Nov. 13, 2018.pdf
Rodney Davis's trip cost:
 Error making sandwich

failed to read James Himes on Nov. 8, 2018 - 

KeyError: 1633

In [13]:
df["Raw Costs"].value_counts()

Raw Costs
Error making sandwich                                                                                                                           311
                                                                                                                                                105
error                                                                                                                                            69
)\n$6,019.15 $2,322.86 $1,223.09 $3,375.78\nTraveler\nAccompanying n/a n/a n/a n/a\n                                                             10
)\n$4,322.73 $2,160.53 $1,035.08 $3,145.01\nTraveler breakdown attached\nAccompanying n/a n/a n/a n/a\n                                           9
                                                                                                                                               ... 
)\n$166.66 ‘ :\nTraveler $1,395.72 $660.00 $88.00 (Visa Processing)\nAccompanying |N/A N/A N/A N/A\n  

In [2]:
df

NameError: name 'df' is not defined

In [19]:
def splitcosts(string):
    if "Accompanying" in string:
        [left, right]=string.split("Accompanying")
        return [left, right]
    else:
        return ["error","error"]
        

In [ ]:
df

In [18]:
df.to_csv("House Travel+Raw Costs Jan 15.csv")

In [16]:
len(broken_links)

66

In [17]:
len(no_forms)

69

In [21]:
df["Raw Traveler Cost"]=df["Raw Costs"].apply(lambda x: splitcosts(x)[0])
df["Raw Accompanying Cost"]=df["Raw Costs"].apply(lambda x:splitcosts(x)[1])

In [4]:
def readcosts(string):
    string=string.replace(",","")
    reg=r"\$\d+"
    numbers=re.findall(reg, string)
    

In [77]:
broken=pd.DataFrame(broken_links)
broken.to_csv("Broken Links2.csv")

In [78]:
no_forms_df=pd.DataFrame(no_forms)
no_forms_df.to_csv("Form Not Found2.csv")

In [79]:
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\..."
1,1.0,1.0,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich
2,2.0,2.0,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich
3,3.0,3.0,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",NaN
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30046,30046.0,46.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30047,30047.0,47.0,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30048,30048.0,48.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN
30049,30049.0,49.0,"Sept. 28, 2007 - Sept. 29, 2007",Geoff Davis,Geoff Davis,R,KY-4,"Alburgueque, NM",Home School Legal Defense Association,Original,https://disclosures-clerk.house.gov/gtimages/M...,USA,,NaN


In [98]:
def readcosts(string):
    reg=r"\$\d+"
    if type(string)==str and "Accompanying" in string:
        traveler, accompanying=string.split("Accompanying")
        traveler=traveler.replace(",","")
        accompanying=accompanying.replace(",","")
        trav_string=re.findall(reg,traveler)
        acc_string=re.findall(reg, accompanying)
        tcosts=[float(x.replace("$","")) for x in trav_string]
        acc_costs=[float(x.replace("$","")) for x in acc_string]
        traveler_costs=sum(tcosts)
        accompanying_costs=sum(acc_costs)
        return [traveler_costs, accompanying_costs]
    else:
        return ["error","error"]
            

In [96]:
#Try that function out
string="apple banana $123.50 $12 $48 Accompanying $530 $650.14"
cleancosts(string)[0]

183.0

In [ ]:
df["Traveler Cost"]=np.nan
df["Accompanying Cost"]=np.nan

In [99]:
df["Traveler Cost"]=df["Raw Cost"].apply(lambda x: readcosts(x)[0])
df["Accompanying Cost"]=df["Raw Cost"].apply(lambda x: readcosts(x)[1])

In [100]:
df

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost,Traveler Cost,Accompanying Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",3440.0,0
1,1.0,1.0,"Dec. 6, 2023 - Dec. 12, 2023",Aniela Butler,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich,error,error
2,2.0,2.0,"Dec. 6, 2023 - Dec. 11, 2023",Brandon Mooney,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,Error making sandwich,Error making sandwich,error,error
3,3.0,3.0,"Dec. 6, 2023 - Dec. 11, 2023",Craig Wheeler,Robert E. Latta,R,OH-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",NaN,error,error
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",30078.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30046,30046.0,46.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurent Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, Massachusetts",BMW Foundation Herbert Quandt,Amendment,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30047,30047.0,47.0,"Sept. 27, 2007 - Sept. 30, 2007",Jasper Pedersen,Robert Wexler,D,FL-19,"Boston, MA",BMW Foundation herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30048,30048.0,48.0,"Sept. 27, 2007 - Sept. 30, 2007",Laurant Crenshaw,Darrell Issa,R,CA-49,"Cape Cod, MA",BMW Foundation Herbert Quandt,Original,https://disclosures-clerk.house.gov/gtimages/S...,USA,,NaN,error,error
30049,30049.0,49.0,"Sept. 28, 2007 - Sept. 29, 2007",Geoff Davis,Geoff Davis,R,KY-4,"Alburgueque, NM",Home School Legal Defense Association,Original,https://disclosures-clerk.house.gov/gtimages/M...,USA,,NaN,error,error


In [101]:
df.to_csv("House Travel+Cleaned Costs.csv")

In [102]:
df[df["Traveler Cost"]!="error"]

,Unnamed: 0.1,Unnamed: 0,Dates,Traveler,Member,Party,State / District,Destination,Sponsor,Filing Type,hrefs,Country,Raw Costs,Raw Cost,Traveler Cost,Accompanying Cost
0,0.0,0.0,"Dec. 6, 2023 - Dec. 12, 2023",Vivian Moeglein,Bruce Westerman,R,AR-4,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",")\n\n$1,738*\n\n$1,702.90 *Averaged increased\...",3440.0,0
4,4.0,4.0,"Dec. 7, 2023 - Dec. 11, 2023",Julia Letlow,Julia Letlow,R,LA-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",")\n\nTraveler $13,703.03 $2,222.00 $451.91\n$1...",30078.0,0
7,7.0,7.0,"Dec. 6, 2023 - Dec. 11, 2023",David Burns,Cathy McMorris Rodgers,R,WA-5,United Arab Emirates,Conservative Climate Foundation,Original,https://disclosures-clerk.house.gov/gtimages/S...,United Arab Emirates,")\nTraveler 1,948.90 $3,050 $1,738* *Averaged ...",")\nTraveler 1,948.90 $3,050 $1,738* *Averaged ...",4788.0,0
11,11.0,11.0,"Dec. 7, 2023 - Dec. 11, 2023",Andrew Garbarino,Andrew Garbarino,R,NY-2,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\nTraveler $13,396.74 $2,222.00 $451.91\n$13,...",")\nTraveler $13,396.74 $2,222.00 $451.91\n$13,...",29123.0,0
13,13.0,13.0,"Dec. 7, 2023 - Dec. 11, 2023",Lori Chavez-DeRemer,Lori Chavez-DeRemer,R,OR-5,United Arab Emirates,Climate Solutions Foundation,Original,https://disclosures-clerk.house.gov/gtimages/M...,United Arab Emirates,")\ndrayeles $10,913.55 $2,222.00 $451.91\n10,3...",")\ndrayeles $10,913.55 $2,222.00 $451.91\n10,3...",13586.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,7405.0,5.0,"Feb. 15, 2019 - Feb. 22, 2019",Judy Chu,Judy Chu,D,CA-27,"Tel Aviv, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\nPl ittached\n$11,549.69 $717.06 Rlease see...","\n\nPl ittached\n$11,549.69 $717.06 Rlease see...",12266.0,0
7423,7423.0,23.0,"Feb. 15, 2019 - Feb. 22, 2019",Michael F. Doyle,Mike Doyle,D,PA-18,"Jerusalem, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\n$2,080.00 $717.06 Please see attached\n\nd...","\n\n$2,080.00 $717.06 Please see attached\n\nd...",2797.0,0
7424,7424.0,24.0,"Feb. 15, 2019 - Feb. 22, 2019",Michael F. Doyle,Mike Doyle,D,PA-18,"Tel Aviv, Israel",J Street Education Fund; Leonard Hill Charitab...,Original,https://disclosures-clerk.house.gov/gtimages/M...,Israel,"\n\n$2,080.00 $717.06 Please see attached\n\nd...","\n\n$2,080.00 $717.06 Please see attached\n\nd...",2797.0,0
7436,7436.0,36.0,"Feb. 13, 2019 - Feb. 17, 2019",Meghan Gallagher,Michael McCaul,R,TX-10,"Abuja, Nigeria",International Republican Institute; National D...,Original,https://disclosures-clerk.house.gov/gtimages/S...,Niger Nigeria,"\n\n$189.86 Abuja,\nKaduna\n\nAccompanying\n","\n\n$189.86 Abuja,\nKaduna\n\nAccompanying\n",189.0,0
